In [1]:
##metaheuristique
import numpy as np
import requests


In [5]:
base_url = "https://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/mknapcb"


for i in range(9):
    url = base_url + str(i+1) + ".txt"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vérifie qu'il n'y a pas d'erreur HTTP
        contenu_texte = response.text

        with open("mknapcb" + str(i+1) + ".txt", "w") as fichier:
            fichier.write(contenu_texte)


    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de l'ouverture de l'URL : {e}")

In [97]:
def extract_data(mon_fichier):
    with open(mon_fichier, "r") as fichier:
        contenu = fichier.read()
        contenu = contenu.split()
        #print(contenu)
        nb_instances = int(contenu[0])
        #print(nb_instances)
        instances = {}
        compteur = 1
        for i in range(1,nb_instances):
            elements = contenu[compteur:]
            n  = int(elements[0])
            #print(n)
            m = int(elements[1])
            #print(m)
            val_opt = float(elements[2])
            cost = np.array([float(elements[e]) for e in range(3,3+n)])
            #print(cost)
            a = np.array([int(elements[e]) for e in range(3+n,3+n+n*m)])
            a = a.reshape(m,n)
            b = np.array([int(elements[e]) for e in range(3+n*m,3+n*m+m)])
            instances[i] = {}
            instances[i]["Nb projet"] = n
            instances[i]["Nb ressource"] = m
            instances[i]["Valeur optimale"] = val_opt
            instances[i]["cost"] = cost
            instances[i]["A"] = a
            instances[i]["B"] = b
            compteur += 3+n+m*n+m
        return nb_instances, instances

In [102]:
mon_fichier = "C:/Users/Thibaud/3A/metaheuristique/Data/mknapcb1.txt"
nb_instances, instances = extract_data(mon_fichier)

a = instances[1]["A"]
cost = instances[1]["Nb ressource"]
cost


5

In [115]:
def heuristique_sac_a_dos(n, m, cost, a, b):
    x= np.zeros(n)
    b_prime = np.sum(b) #somme des ressources
    #print(b)
    a_prime= np.sum(a, axis=0) #somme des ressources nécessaires pour chaque projet
    #print(a)
    y = -cost/a_prime #signe - pour trier dans l'ordre décroissant
    #print(y)
    indices =y.argsort()
    #print(indices)
    ressource = 0
    index = []
    for i in indices:
        if ressource + a_prime[i] <= b_prime:
            x[i] = 1
            ressource += a_prime[i]
            index.append(i) 
    #verification solution réalisable
    last_index = index[-1]
    while any(np.dot(a, x) > b):
        x[last_index] = 0
        index.pop()

    return x

In [116]:
c = np.array([1,1,5,3])
a = np.array([[1,1,1,1],[2,1,3,1]])
b= np.array([1,4])
n=4
m=2

heuristique_sac_a_dos(n, m, c, a, b)

array([0., 0., 0., 1.])